In [2]:
!pip install pytrends plotly pandas

In [3]:
import pandas as pd

#Basic Setup
`.build_payload()` prepares the query to Google Trends before you can fetch any data. It tells Google which keywords you’re interested in, the timeframe, the region, the category, and the type of search (web, YouTube, news, etc.).


| Parameter   | Description                                                                                                 |
| ----------- | ----------------------------------------------------------------------------------------------------------- |
| `kw_list`   | List of search terms (e.g., `['almond milk']`) — required                                                   |
| `cat`       | Google category ID (default `0` = all categories)                                                           |
| `timeframe` | Time period (e.g., `'now 7-d'`, `'today 5-y'`, `'all'`)                                                     |
| `geo`       | Geographic region (e.g., `'US'`, `'CA'`, `'IN'`)                                                            |
| `gprop`     | Type of search:<br>`''` = web search (default)<br>`'images'`, `'news'`, `'youtube'`, `'froogle'` (shopping) |


In [4]:
from pytrends.request import TrendReq

# Connect to Google
pytrends = TrendReq(hl='en-US', tz=360)  # US English, UTC-6

# Example keyword
pytrends.build_payload(['almond milk'], cat=0, timeframe='today 12-m', geo='US', gprop='')

#Potentially Useful Methods - Google Trends API
| **Method**            | **Description**                                                     | **Returns**                                                              | **Use Case for Dashboard**                                                                |
| ---------------------- | ------------------------------------------------------------------- | ------------------------------------------------------------------------ | ----------------------------------------------------------------------------------------- |
| `interest_over_time()` | Returns trend data (normalized 0–100) over time                     | DataFrame with timestamps, search interest scores, and `isPartial` flags | Forecast demand trends over the year for a product/ingredient (e.g. “matcha”, “oat milk”) |
| `interest_by_region()` | Trend interest by state, city, or country                           | DataFrame with regions as rows and interest scores per keyword           | Identify regional demand patterns for a CPG brand                                         |
| `related_queries()`    | What people search alongside your term                              | Dictionary with `'top'` and `'rising'` DataFrames per keyword            | Identify new flavors, competitors, or co-branded ideas                                    |
| `trending_searches()`  | Daily trending searches                                             | DataFrame of currently hot topics by region                              | Spot viral topics that may impact a category                                              |
| `top_charts()`         | Top searches in a given year and category                           | DataFrame of most searched items in a year                               | Historical top trends (great for year-end or long-term trend dashboards)                  |
| `suggestions()`        | Autocomplete suggestions for refining keyword search                | List of dictionaries with suggested search terms                         | Generate variants (e.g., “oat milk vs oat creamer”)                                       |
| `categories()`         | Category IDs used in filtering queries (e.g., Food & Drink, Beauty) | Dictionary of nested category names and numeric IDs                      | Segment trends by CPG-relevant categories (e.g., beverages, snacks, cosmetics)            |


In [5]:
#Example: Trend Over Time for ‘oat milk’
pytrends.build_payload(['oat milk'], timeframe='today 12-m', geo='US')
data = pytrends.interest_over_time()
data.head()

,oat milk,isPartial
date,,
2024-06-16,86,False
2024-06-23,84,False
2024-06-30,83,False
2024-07-07,81,False
2024-07-14,85,False


In [6]:
#Example: Interest by Region
pytrends.build_payload(['oat milk'], timeframe='today 12-m', geo='US')
region_df = pytrends.interest_by_region(resolution='STATE', inc_low_vol=True)
region_df.sort_values(by='oat milk', ascending=False).head()

,oat milk
geoName,
California,100
Vermont,95
District of Columbia,93
Massachusetts,88
Washington,86


In [7]:
from pytrends.request import TrendReq
import pandas as pd
import plotly.express as px

# Step 1: Keyword and pytrends
keyword = 'oat milk'
pytrends = TrendReq()
pytrends.build_payload([keyword], geo='US', timeframe='today 12-m')

# Step 2: Get regional interest
df = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True).reset_index()
df.columns = df.columns.str.strip()
df = df.rename(columns={keyword: 'Interest'})

# Step 3: Add state abbreviations
us_states = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['state_code'] = df['geoName'].map(us_states)

# Step 4: Plot
fig = px.choropleth(
    df,
    locations='state_code',
    locationmode='USA-states',
    scope='usa',
    color='Interest',
    hover_name='geoName',
    color_continuous_scale='Blues',
    labels={'Interest': f'Interest in "{keyword}"'},
    title=f'Google Trends Interest in "{keyword}" by U.S. State'
)
fig.show()

In [8]:
# Step 1: Keyword and pytrends
keyword = 'matcha'
pytrends = TrendReq()
pytrends.build_payload([keyword], geo='US', timeframe='today 12-m')

# Step 2: Get regional interest
df = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True).reset_index()
df.columns = df.columns.str.strip()
df = df.rename(columns={keyword: 'Interest'})

# Step 3: Add state abbreviations
us_states = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['state_code'] = df['geoName'].map(us_states)

# Step 4: Plot
fig = px.choropleth(
    df,
    locations='state_code',
    locationmode='USA-states',
    scope='usa',
    color='Interest',
    hover_name='geoName',
    color_continuous_scale='Blues',
    labels={'Interest': f'Interest in "{keyword}"'},
    title=f'Google Trends Interest in "{keyword}" by U.S. State'
)
fig.show()

In [9]:
# Step 1: Keyword and pytrends
keyword = 'In-N-Out'
pytrends = TrendReq()
pytrends.build_payload([keyword], geo='US', timeframe='today 12-m')

# Step 2: Get regional interest
df = pytrends.interest_by_region(resolution='REGION', inc_low_vol=True).reset_index()
df.columns = df.columns.str.strip()
df = df.rename(columns={keyword: 'Interest'})

# Step 3: Add state abbreviations
us_states = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR', 'California': 'CA',
    'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE', 'Florida': 'FL', 'Georgia': 'GA',
    'Hawaii': 'HI', 'Idaho': 'ID', 'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA',
    'Kansas': 'KS', 'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV', 'New Hampshire': 'NH',
    'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY', 'North Carolina': 'NC',
    'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK', 'Oregon': 'OR', 'Pennsylvania': 'PA',
    'Rhode Island': 'RI', 'South Carolina': 'SC', 'South Dakota': 'SD', 'Tennessee': 'TN',
    'Texas': 'TX', 'Utah': 'UT', 'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA',
    'West Virginia': 'WV', 'Wisconsin': 'WI', 'Wyoming': 'WY'
}
df['state_code'] = df['geoName'].map(us_states)

# Step 4: Plot
fig = px.choropleth(
    df,
    locations='state_code',
    locationmode='USA-states',
    scope='usa',
    color='Interest',
    hover_name='geoName',
    color_continuous_scale='Blues',
    labels={'Interest': f'Interest in "{keyword}"'},
    title=f'Google Trends Interest in "{keyword}" by U.S. State'
)
fig.show()

In [10]:
pip install google-api-python-client

Note: you may need to restart the kernel to use updated packages.


#Potentially Useful Methods - Youtube API
| **Method**                 | **Returns**                                                          | **Use Case**                                                                    |
| -------------------------- | -------------------------------------------------------------------- | ------------------------------------------------------------------------------- |
| `search().list()`          | Metadata for videos, channels, or playlists matching a query         | - Keyword search<br>- Find trending or recent videos<br>- Filter by region/date |
| `videos().list()`          | Details for one or more videos: title, views, likes, duration, etc.  | - Video analytics<br>- Engagement tracking<br>- Content enrichment              |
| `commentThreads().list()`  | Top-level comments on a video (text, likes, publish time, author)    | - Sentiment analysis<br>- User feedback mining<br>- Comment scraping            |
| `channels().list()`        | Channel info: subscriber count, total views, uploads playlist, etc.  | - Creator insights<br>- Competitor analysis<br>- Channel tracking               |
| `playlistItems().list()`   | All videos in a given playlist (e.g., uploads, curated lists)        | - Get full video lists<br>- Scrape all uploads<br>- Analyze playlists           |
| `videoCategories().list()` | List of video categories (e.g., Music, Education) by region          | - Category tagging<br>- Data labeling<br>- Content classification               |
| `videos().getRating()`     | Authenticated user's rating of a video (like/dislike/none)           | - Personalized apps (requires OAuth)<br>- User engagement tracking              |
| `activities().list()`      | A channel’s activity feed: uploads, likes, comments (OAuth required) | - Track user/channel behavior<br>- Social graph exploration                     |

In [11]:
from googleapiclient.discovery import build

api_key = 'API-KEY'
youtube = build('youtube', 'v3', developerKey=api_key)

# Step 2: Search for videos related to "oat milk"
search_response = youtube.search().list(
    q='oat milk',
    part='snippet',
    type='video',
    maxResults=5  # Returns 5 videos
).execute()

# Step 3: Extract video IDs
video_ids = [item['id']['videoId'] for item in search_response['items']]

# Step 4: Fetch video statistics (view count)
video_response = youtube.videos().list(
    part='snippet,statistics',
    id=','.join(video_ids)
).execute()

# Step 5: Print titles and view counts
for item in video_response['items']:
    title = item['snippet']['title']
    views = item['statistics'].get('viewCount', '0')
    print(f"{title} — {int(views):,} views")


HttpError: <HttpError 400 when requesting https://youtube.googleapis.com/youtube/v3/search?q=oat+milk&part=snippet&type=video&maxResults=5&key=API-KEY&alt=json returned "API key not valid. Please pass a valid API key.". Details: "[{'message': 'API key not valid. Please pass a valid API key.', 'domain': 'global', 'reason': 'badRequest'}]">

In [ ]:
# Step 1: Search for videos related to the chosen produt
search_response = youtube.search().list(
    q='Buldak Ramen',
    part='snippet',
    type='video',
    maxResults=25  # Fetch more to get a better top 5
).execute()

# Step 2: Extract video IDs
video_ids = [item['id']['videoId'] for item in search_response['items']]

# Step 3: Get statistics for each video
video_response = youtube.videos().list(
    part='snippet,statistics',
    id=','.join(video_ids)
).execute()

# Step 4: Sort by view count
videos = []
for video in video_response['items']:
    title = video['snippet']['title']
    views = int(video['statistics'].get('viewCount', 0))
    videos.append((title, views))

# Sort descending by views
top_videos = sorted(videos, key=lambda x: x[1], reverse=True)[:5]

# Step 5: Print top 5
for i, (title, views) in enumerate(top_videos, 1):
    print(f"{i}. {title} — {views:,} views")

1. carbonara buldak ramen tteokbokki #buldak #buldakramen #buldaknoodles #conveniencestorefood — 27,780,357 views
2. Ranking EVERY Flavor Of Buldak Ramen! — 21,082,736 views
3. carbo buldak ramen with buldak mac and cheese gratin #asmr #koreanfood — 16,735,644 views
4. Ranking The RAREST Buldak Flavors! — 16,153,888 views
5. trying to SURVIVE buldak carbonara ramen #short #buldak — 10,141,471 views


Getting the top videos for a topic and looking at the view counts can help predict trends. For example, the "Buldak Ramen" videos have view counts in the millions while the "oat milk" videos have view counts in the thousands. This aligns with our expectations and confirms that Buldak Ramen is a more trending product on Youtube.